In [194]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import time
import re
import time
import random
import os

In [195]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, embedding_dim, hidden_size, dropout=0.3, num_layers=2):
        super().__init__()
        
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.dropout_rate = dropout
        self.num_layers = num_layers
        
        self.embedding = None
        self.lstm = None
        self.dropout = None
        self.output_layer = None
        self.vocab_size = None
        self.criterion = None
        
        # Dictionnaires
        self.word_to_idx = {}
        self.idx_to_word = {}

    def _initialize_layers(self):
        if self.vocab_size is None:
            raise ValueError("Vous devez d'abord appeler build_vocabulary()")
        
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim, padding_idx=0)
        
        # bidirectional=False pour lire dans 1 sens
        self.lstm = nn.LSTM(
            self.embedding_dim, 
            self.hidden_size, 
            num_layers=self.num_layers,
            batch_first=True, 
            dropout=self.dropout_rate if self.num_layers > 1 else 0,
            bidirectional=False 
        )
        
        self.dropout = nn.Dropout(self.dropout_rate)
        
        self.output_layer = nn.Linear(self.hidden_size, self.vocab_size)
        
        self.criterion = nn.CrossEntropyLoss(ignore_index=0)
        
        print(f"Modèle prêt : vocab={self.vocab_size}, hidden={self.hidden_size}, Unidirectionnel")
    
    def forward(self, x, hidden=None):
        # On permet de passer un état caché (hidden) pour la génération efficace
        embedded = self.embedding(x)
        embedded = self.dropout(embedded)
        
        # Si hidden est fourni, on l'utilise, sinon LSTM l'initialise à 0
        lstm_out, hidden = self.lstm(embedded, hidden)
        
        lstm_out = self.dropout(lstm_out)
        output = self.output_layer(lstm_out)
        
        # On retourne aussi l'état caché pour la prochaine étape de génération
        return output, hidden
    
    def build_vocabulary(self, texts):
        special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
        
        word_counts = {}
        for text in texts:
            # Simple tokenization par espace
            for word in text.split():
                word_counts[word] = word_counts.get(word, 0) + 1
        
        all_words = sorted([w for w, c in word_counts.items() if c >= 2])
        final_vocab = special_tokens + all_words
        
        self.word_to_idx = {word: i for i, word in enumerate(final_vocab)}
        self.idx_to_word = {i: word for i, word in enumerate(final_vocab)}
        
        self.vocab_size = len(final_vocab)
        print(f"Vocabulaire créé : {self.vocab_size} mots")
        
        self._initialize_layers()
    
    def text_to_indices(self, text):
        words = text.split()
        sequence = ["<BOS>"] + words + ["<EOS>"]
        return [self.word_to_idx.get(word, self.word_to_idx["<UNK>"]) for word in sequence]
    
    def indices_to_text(self, indices):
        words = []
        for idx in indices:
            word = self.idx_to_word.get(idx, "")
            if word not in ["<BOS>", "<EOS>", "<PAD>", "<UNK>", ""]:
                words.append(word)
        result = " ".join(words)
        result = re.sub(r"\s+'", "'", result)
        result = re.sub(r"'\s+", "'", result)
        return result

    def pad_sequence(self, indices, max_length):
        if len(indices) >= max_length:
            return indices[:max_length]
        return indices + [0] * (max_length - len(indices))

    def create_training_pair(self, sequence):
        # Gestion du cas où sequence ne contient pas EOS (juste par sécurité)
        if self.word_to_idx["<EOS>"] in sequence:
            eos_idx = sequence.index(self.word_to_idx["<EOS>"])
            input_seq = sequence[:eos_idx]
            target_seq = sequence[1:eos_idx + 1]
        else:
            # Fallback si pas de EOS
            input_seq = sequence[:-1]
            target_seq = sequence[1:]
        return input_seq, target_seq

    def prepare_batch(self, texts):
        all_inputs = []
        all_targets = []
        
        for text in texts:
            indices = self.text_to_indices(text)
            if len(indices) < 2: continue # Skip trop court
            input_seq, target_seq = self.create_training_pair(indices)
            all_inputs.append(input_seq)
            all_targets.append(target_seq)
        
        if not all_inputs: return None, None

        max_len = max(len(seq) for seq in all_inputs)
        padded_inputs = [self.pad_sequence(seq, max_len) for seq in all_inputs]
        padded_targets = [self.pad_sequence(seq, max_len) for seq in all_targets]
        
        return torch.tensor(padded_inputs), torch.tensor(padded_targets)

    def train_step(self, input_batch, target_batch):
        # Le forward retourne un tuple (output, hidden)
        predictions, _ = self(input_batch) 
        
        predictions_flat = predictions.view(-1, self.vocab_size)
        targets_flat = target_batch.view(-1)
        
        loss = self.criterion(predictions_flat, targets_flat)
        return loss
    
    def generate_text(self, start_text, max_length=20, temperature=1.0, top_k=10, repetition_penalty=1.2):
        self.eval()
        
        # Récupération du device (CPU, CUDA ou MPS) pour éviter les erreurs de tenseur
        device = next(self.parameters()).device
        
        current_seq = self.text_to_indices(start_text)
        
        # Retrait du token EOS s'il est présent à la fin
        if current_seq and current_seq[-1] == self.word_to_idx.get("<EOS>"):
            current_seq = current_seq[:-1]
            
        generated_words = []
        hidden = None
        
        # Conversion initiale sur le bon device
        input_tensor = torch.tensor([current_seq], device=device)
        
        with torch.no_grad():
            output, hidden = self(input_tensor, hidden)
            next_token_logits = output[0, -1, :]

        for _ in range(max_length):
            logits = next_token_logits.clone()
            
            # 1. Bloquer tokens spéciaux
            for token in ["<PAD>", "<UNK>", "<BOS>"]:
                if token in self.word_to_idx:
                    logits[self.word_to_idx[token]] = float('-inf')

            # 2. Pénalité de répétition (fenêtre de 5 mots)
            for word_idx in generated_words[-5:]:
                if logits[word_idx] > 0:
                    logits[word_idx] /= repetition_penalty
                else:
                    logits[word_idx] *= repetition_penalty
            
            # 3. RÈGLES GRAMMATICALES FORCÉES (Logic Patching)
            if generated_words:
                last_idx = generated_words[-1]
                last_word = self.idx_to_word.get(last_idx, "")
                
                # Règle : Après "que je", on évite l'indicatif (suis, vais, sais...)
                if last_word == "je" and len(generated_words) >= 2:
                    prev_idx = generated_words[-2]
                    prev_word = self.idx_to_word.get(prev_idx, "")
                    
                    if prev_word == "que":
                        # On réduit drastiquement la probabilité de l'indicatif
                        indicatifs = ["suis", "sais", "vais", "peux", "veux", "ai"]
                        for verb in indicatifs:
                            if verb in self.word_to_idx:
                                logits[self.word_to_idx[verb]] -= 5.0 # Pénalité forte (soustraction sur les logits)
            
            # Sampling
            logits = logits / temperature
            top_k_actual = min(top_k, self.vocab_size)
            filtered_logits, top_k_indices = torch.topk(logits, top_k_actual)
            probs = torch.softmax(filtered_logits, dim=-1)
            
            predicted_idx_in_topk = torch.multinomial(probs, 1).item()
            predicted_word_idx = top_k_indices[predicted_idx_in_topk].item()
            
            if predicted_word_idx == self.word_to_idx.get("<EOS>"):
                break
            
            generated_words.append(predicted_word_idx)
            
            # Préparation itération suivante
            input_tensor = torch.tensor([[predicted_word_idx]], device=device)
            with torch.no_grad():
                output, hidden = self(input_tensor, hidden)
                next_token_logits = output[0, -1, :]
        
        self.train()
        return self.indices_to_text(current_seq + generated_words)

    def generate_best(self, prompt, n_candidates=5, max_length=15, temperature=0.7, top_k=15):
        """Génère plusieurs candidats et retourne le plus cohérent selon des critères heuristiques"""
        candidates = []
        
        for _ in range(n_candidates):
            text = self.generate_text(prompt, max_length, temperature, top_k)
            candidates.append(text)
        
        def score_coherence(text):
            score = 0
            words = text.split()
            
            # Critère 1: Longueur (pour éviter les trucs trop courts)
            if len(words) < 4: score -= 10
            
            # Critère 2: Répétitions strictes
            if len(words) != len(set(words)): score -= 5
            
            # Critère 3: Fin de phrase abrupte (prépositions/articles à la fin)
            if words[-1] in ["le", "la", "les", "un", "une", "de", "à", "que", "qui", "pour", "et"]:
                score -= 8
            
            # Critère 4: Erreurs grammaticales connues (Correction des logs précédents)
            bad_phrases = [
                "faut que je suis", "faut que je sais", "faut que je vais", # Subjonctif incorrect
                "faut que je me suis", 
                "acheter une heure", # Non-sens sémantique
                "parler à moi", # Syntaxique (me parler)
                "sais pas le", 
                "démontrer vu"
            ]
            
            for bad in bad_phrases:
                if bad in text:
                    score -= 20  # Grosse pénalité
            
            return score
        
        # On trie les candidats du meilleur score au pire
        candidates.sort(key=score_coherence, reverse=True)
        
        # (Optionnel) Afficher les candidats pour debug
        # print(f"--- Debug '{prompt}' ---")
        # for c in candidates:
        #     print(f"Score {score_coherence(c)}: {c}")
            
        return candidates[0]

Chargement et Nettoyage du Corpus 

In [196]:
import os
import re

print("="*60)
print("CHARGEMENT ET NETTOYAGE DU CORPUS")
print("="*60)

csv_path = "/Users/guillaume/Docs/Formations/Intro-Deep-Learning/sentences.csv"

def clean_sentence(sentence):
    sentence = sentence.lower().strip()
    
    # 1. Exclure les phrases problématiques
    excluded = ['muiriel', 'www', 'http', '@', '#', '...', '..', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    if any(ex in sentence for ex in excluded):
        return None
    
    # 2. Gestion intelligente de la ponctuation pour le français
    # On garde les apostrophes mais on les sépare, et on vire le reste
    # Cette regex remplace "l'école" par "l' école" et "j'aime" par "j' aime"
    sentence = re.sub(r"(['])", r"\1 ", sentence) 
    
    # Suppression de la ponctuation (sauf apostrophes qu'on a traitées)
    # On garde aussi les tirets pour les mots composés (ex: arc-en-ciel)
    sentence = re.sub(r'[«»""„\.\,\!\?\;\:\(\)\[\]\{\}]', ' ', sentence)
    
    # Réduction des espaces multiples
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    
    words = sentence.split()
    
    # 3. Filtre de longueur
    # 5000 phrases c'est peu, donc on reste sur des phrases courtes à moyennes
    if not (4 <= len(words) <= 15): 
        return None
    
    return ' '.join(words)

if os.path.exists(csv_path):
    french_sentences = []
    seen = set()
    
    with open(csv_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 3:
                lang_code = parts[1]
                sentence = parts[2]
                
                if lang_code == 'fra':
                    cleaned = clean_sentence(sentence)
                    
                    if cleaned and cleaned not in seen:
                        seen.add(cleaned)
                        french_sentences.append(cleaned)
                
                if len(french_sentences) >= 100000:
                    break
    
    texts = french_sentences
    
    print(f"✓ Chargé {len(texts)} phrases françaises")
    print(f"\nExemples:")
    for i in range(100):
        print(f"  {i+1}. {texts[i]}")
else:
    print(f"⚠ Fichier non trouvé!")
    texts = []

CHARGEMENT ET NETTOYAGE DU CORPUS
✓ Chargé 100000 phrases françaises

Exemples:
  1. je ne supporte pas ce type
  2. ne tenez aucun compte de ce qu' il dit
  3. qu' est-ce que tu fais
  4. qu' est-ce que c' est
  5. je serai bientôt de retour
  6. je ne sais pas
  7. j' en perds mes mots
  8. ça ne va jamais finir
  9. c’était un méchant lapin
  10. j' étais dans les montagnes
  11. est-ce que c' est une photo récente
  12. je ne sais pas si j' ai le temps
  13. pour une certaine raison le microphone ne marchait pas tout à l' heure
  14. tout le monde doit apprendre par soi-même en fin de compte
  15. l' éducation dans ce monde me déçoit
  16. l' apprentissage ne devrait pas être forcé l' apprentissage devrait être encouragé
  17. ça ne changera rien
  18. il se peut que j' abandonne bientôt et fasse une sieste à la place
  19. c' est parce que tu ne veux pas être seul
  20. ça n' arrivera pas
  21. parfois il peut être un gars bizarre
  22. je ferai de mon mieux pour ne pas perturber 

Configuration sur 5k phrases

In [ ]:
# --- 1. CONFIGURATION ---
# Détection automatique GPU/MPS (Mac M1/M2)/CPU
device = torch.device("cpu")
print(f"Utilisation forcé du device : {device} (pour stabilité Embedding)")

if len(texts) > 0:
    print("\n" + "="*60)
    print("CONFIGURATION DU MODÈLE")
    print("="*60)

    # embedding 256 / hidden 256 est suffisant pour < 100k phrases
    model = LSTMLanguageModel(
        embedding_dim=256,
        hidden_size=256, 
        dropout=0.4,       # Augmenté un peu pour forcer la généralisation
        num_layers=2
    ).to(device)
    
    model.build_vocabulary(texts)
    
    # Optimiseur et Scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.002, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )
    
    # Préparation globale des données
    # Note : Idéalement faire le padding par batch, mais ici simplifié
    full_inputs, full_targets = model.prepare_batch(texts)
    
    # Création d'un Dataset et DataLoader pour gérer les mini-batchs facilement
    from torch.utils.data import TensorDataset, DataLoader
    
    dataset = TensorDataset(full_inputs, full_targets)
    # batch_size=64 est un bon standard (32 si manque de RAM, 128 si GPU puissant)
    batch_size = 64 
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    print(f"\nDonnées totales: {len(full_inputs)} phrases")
    print(f"Taille des batchs: {batch_size}")
    print(f"Nombre de batchs par époque: {len(dataloader)}")
    print(f"Paramètres du modèle: {sum(p.numel() for p in model.parameters()):,}")

    print("\n=== ENTRAÎNEMENT AVEC MINI-BATCHS ===")
    
    model.train()
    num_epochs = 100
    
    history_loss = []

    try:
        for epoch in range(num_epochs):
            start_time = time.time()
            epoch_loss = 0
            
            for batch_inputs, batch_targets in dataloader:
                # Envoi des batchs sur le device (GPU/CPU)
                batch_inputs = batch_inputs.to(device)
                batch_targets = batch_targets.to(device)
                
                optimizer.zero_grad()
                
                # Forward & Loss
                loss = model.train_step(batch_inputs, batch_targets)
                
                # Backward
                loss.backward()
                
                # Clipping pour éviter l'explosion des gradients (important pour LSTM)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                
                epoch_loss += loss.item()
            
            # Calcul de la moyenne de la loss pour cette époque
            avg_loss = epoch_loss / len(dataloader)
            history_loss.append(avg_loss)
            
            # Le scheduler ajuste le learning rate en fonction de la loss moyenne
            scheduler.step(avg_loss)
            
            # Affichage périodique
            if (epoch + 1) % 10 == 0 or epoch == 0:
                lr = optimizer.param_groups[0]['lr']
                elapsed = time.time() - start_time
                print(f"Epoch {epoch+1:3d}/{num_epochs} | Loss: {avg_loss:.4f} | LR: {lr:.6f} | Temps: {elapsed:.2f}s")
                
                # Petit test de génération en direct pour voir les progrès !
                if (epoch + 1) % 20 == 0:
                    print(f"   >>> Génération test : {model.generate_text('je suis', max_length=10)}")

    except KeyboardInterrupt:
        print("\nEntraînement interrompu manuellement.")

    print(f"\n{'='*60}")
    print(f"ENTRAÎNEMENT TERMINÉ - Loss finale: {history_loss[-1]:.4f}")
    print(f"{'='*60}")

Configuration sur 100k phrases

In [197]:
# ==========================================
# 1. CONFIGURATION
# ==========================================
FILE_PATH = "fra_sentences.tsv"  # <--- CHEMIN COMPLET ICI SI BESOIN
LIMIT_SENTENCES = 100000         # Objectif : 100k phrases
BATCH_SIZE = 64
EMBEDDING_DIM = 256              
HIDDEN_SIZE = 512                
NUM_LAYERS = 2
DROPOUT = 0.5                    # Fort dropout pour éviter le par-cœur
LEARNING_RATE = 0.001
EPOCHS = 20

# Force CPU pour éviter le bug MPS (Embedding)
device = torch.device("cpu")
print(f" Configuration : Device={device} | Target={LIMIT_SENTENCES} phrases")

# ==========================================
# 2. FONCTIONS DE NETTOYAGE & CHARGEMENT
# ==========================================
def clean_sentence_robust(sentence):
    sentence = sentence.lower().strip()
    # Exclure caractères bizarres et chiffres isolés
    if any(c in sentence for c in ['@', '#', 'www', 'http', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']):
        return None
    # Séparer les apostrophes (l'école -> l' école)
    sentence = re.sub(r"(['])", r"\1 ", sentence)
    # Nettoyer ponctuation
    sentence = re.sub(r'[«»""„\.\,\!\?\;\:\(\)\[\]\{\}]', ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    # Filtre longueur (ni trop court, ni trop long)
    words = sentence.split()
    if not (4 <= len(words) <= 18): 
        return None
    return ' '.join(words)

def load_tatoeba(filepath, limit):
    print(f" Lecture de {filepath}...")
    sentences = []
    if not os.path.exists(filepath):
        print(f" ERREUR : Fichier introuvable à : {filepath}")
        return []
    
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            # Tatoeba format : ID \t LANG \t TEXT
            # Vérifier que c'est bien du français ('fra')
            if len(parts) == 3 and parts[1] == 'fra':
                text = parts[2]
                cleaned = clean_sentence_robust(text)
                if cleaned:
                    sentences.append(cleaned)
            
            if len(sentences) >= limit:
                break
    
    print(f" Chargé {len(sentences)} phrases propres.")
    return sentences

# ==========================================
# 3. PRÉPARATION DES DONNÉES
# ==========================================
texts = load_tatoeba(FILE_PATH, LIMIT_SENTENCES)

if len(texts) > 0:
    # Train / Validation Split (90% / 10%)
    random.seed(42)
    random.shuffle(texts)
    split_idx = int(len(texts) * 0.9)
    train_texts = texts[:split_idx]
    val_texts = texts[split_idx:]
    
    print(f" Train: {len(train_texts)} | Validation: {len(val_texts)}")

    # Initialisation du modèle 
    model = LSTMLanguageModel(EMBEDDING_DIM, HIDDEN_SIZE, DROPOUT, NUM_LAYERS).to(device)
    model.build_vocabulary(train_texts)

    # Création des DataLoaders
    train_inputs, train_targets = model.prepare_batch(train_texts)
    val_inputs, val_targets = model.prepare_batch(val_texts)

    train_data = TensorDataset(train_inputs, train_targets)
    val_data = TensorDataset(val_inputs, val_targets)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=BATCH_SIZE) # Pas besoin de shuffle pour la validation

    # Optimiseur
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # ==========================================
    # 4. BOUCLE D'ENTRAÎNEMENT (TRAIN + VAL)
    # ==========================================
    print(f"\n Démarrage de l'entraînement ({EPOCHS} époques)...")
    best_val_loss = float('inf')
    
    history_train = []
    history_val = []

    try:
        for epoch in range(EPOCHS):
            start_time = time.time()
            
            # --- PHASE TRAIN ---
            model.train()
            total_train_loss = 0
            
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                
                optimizer.zero_grad()
                loss = model.train_step(batch_x, batch_y)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                
                total_train_loss += loss.item()
            
            avg_train_loss = total_train_loss / len(train_loader)
            
            # --- PHASE VALIDATION ---
            model.eval()
            total_val_loss = 0
            with torch.no_grad():
                for batch_x, batch_y in val_loader:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                    # Calcul manuel de la loss pour la validation
                    predictions, _ = model(batch_x)
                    # Redimensionner (view) pour que la Loss function accepte les données
                    loss = model.criterion(predictions.view(-1, model.vocab_size), batch_y.view(-1))
                    total_val_loss += loss.item()
            
            avg_val_loss = total_val_loss / len(val_loader)
            
            # --- LOGS & SAVE ---
            history_train.append(avg_train_loss)
            history_val.append(avg_val_loss)
            scheduler.step(avg_val_loss)
            
            duration = time.time() - start_time
            print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | {duration:.0f}s")
            
            # Sauvegarde si record battu
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'vocab_word_to_idx': model.word_to_idx,
                    'vocab_idx_to_word': model.idx_to_word,
                    'vocab_size': model.vocab_size,
                    'embedding_dim': EMBEDDING_DIM,
                    'hidden_size': HIDDEN_SIZE,
                    'num_layers': NUM_LAYERS,
                    'dropout': DROPOUT
                }, "best_model_100k.pth")
                print("   ★ Nouveau meilleur modèle sauvegardé !")
            
            # Test intermédiaire (toutes les 5 époques)
            if (epoch+1) % 5 == 0:
                print(f"   >>> Test: {model.generate_best('je suis', max_length=10)}")
    except KeyboardInterrupt:
        print("\n Entraînement stoppé manuellement.")

 Configuration : Device=cpu | Target=100000 phrases
 Lecture de fra_sentences.tsv...
 Chargé 100000 phrases propres.
 Train: 90000 | Validation: 10000
Vocabulaire créé : 16434 mots
Modèle prêt : vocab=16434, hidden=512, Unidirectionnel

 Démarrage de l'entraînement (20 époques)...
Epoch 01 | Train Loss: 5.3411 | Val Loss: 4.5660 | 332s
   ★ Nouveau meilleur modèle sauvegardé !
Epoch 02 | Train Loss: 4.6411 | Val Loss: 4.2936 | 1211s
   ★ Nouveau meilleur modèle sauvegardé !
Epoch 03 | Train Loss: 4.4164 | Val Loss: 4.1610 | 272s
   ★ Nouveau meilleur modèle sauvegardé !
Epoch 04 | Train Loss: 4.2758 | Val Loss: 4.0784 | 194s
   ★ Nouveau meilleur modèle sauvegardé !
Epoch 05 | Train Loss: 4.1744 | Val Loss: 4.0245 | 183s
   ★ Nouveau meilleur modèle sauvegardé !
   >>> Test: je suis désolé de ne pas te voir
Epoch 06 | Train Loss: 4.0925 | Val Loss: 3.9892 | 167s
   ★ Nouveau meilleur modèle sauvegardé !
Epoch 07 | Train Loss: 4.0272 | Val Loss: 3.9629 | 429s
   ★ Nouveau meilleur modèl

In [198]:
print("=== TEST AVEC PROMPTS DU CORPUS ===\n")

# Prompts plus probables dans Tatoeba
test_prompts = [
    "je vais",
    "il est",
    "elle a", 
    "nous avons",
    "c'est une",
    "je pense que"
]

# On s'assure que le modèle est en mode évaluation (pas de dropout)
model.eval()

for prompt in test_prompts:
    
    generated = model.generate_text(
        prompt, 
        max_length=10,      
        temperature=0.4,    
        top_k=10
    )
    print(f"'{prompt}' → {generated}")

=== TEST AVEC PROMPTS DU CORPUS ===

'je vais' → je vais vous voir demain
'il est' → il est très gentil
'elle a' → elle a l'air d'être malade
'nous avons' → nous avons des difficultés à l'école
'c'est une' → une autre fois
'je pense que' → je pense que vous êtes très gentille


In [199]:
import torch

# Vérifier que le modèle existe bien en mémoire avant de sauvegarder
if 'model' in globals():
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        # Sauvegarder tout ce qu'il faut pour reconstruire l'architecture
        'vocab_word_to_idx': model.word_to_idx,
        'vocab_idx_to_word': model.idx_to_word,
        'vocab_size': model.vocab_size,
        'embedding_dim': model.embedding_dim,
        'hidden_size': model.hidden_size,
        'num_layers': model.num_layers,
        'dropout': model.dropout_rate
    }
    
    # C'est CE fichier qu'il faudra charger
    torch.save(checkpoint, "mon_modele_lstm_francais.pth")
    print("Modèle sauvegardé sous 'mon_modele_lstm_francais.pth'")
else:
    print("Erreur : Le modèle n'est pas en mémoire. Tu dois ré-entraîner avant de sauvegarder.")

Modèle sauvegardé sous 'mon_modele_lstm_francais.pth'


In [200]:
import torch
import os

# Nom du fichier qu'on vient de créer
filename = "mon_modele_lstm_francais.pth"

if os.path.exists(filename):
    # CHARGEMENT CORRIGÉ
    # On précise weights_only=False car on charge aussi la structure du vocabulaire (dictionnaires)
    checkpoint = torch.load(filename, map_location=torch.device('cpu'), weights_only=False)

    print("Paramètres trouvés :")
    print(f"- Embed: {checkpoint['embedding_dim']}")
    print(f"- Hidden: {checkpoint['hidden_size']}")
    print(f"- Vocab: {checkpoint['vocab_size']}")

    # 1. Création de l'instance vide
    loaded_model = LSTMLanguageModel(
        embedding_dim=checkpoint['embedding_dim'],
        hidden_size=checkpoint['hidden_size'],
        num_layers=checkpoint['num_layers'],
        dropout=checkpoint['dropout']
    )

    # 2. Restauration du vocabulaire
    loaded_model.word_to_idx = checkpoint['vocab_word_to_idx']
    loaded_model.idx_to_word = checkpoint['vocab_idx_to_word']
    loaded_model.vocab_size = checkpoint['vocab_size']
    
    # Important : initialiser les couches AVANT de charger les poids
    loaded_model._initialize_layers() 

    # 3. Chargement des poids
    loaded_model.load_state_dict(checkpoint['model_state_dict'])
    loaded_model.eval()

    print("\n Modèle chargé et prêt !")
    print(f"Test génération : {loaded_model.generate_best('je suis')}")

else:
    print(f" Le fichier '{filename}' est introuvable. As-tu lancé l'étape de sauvegarde ?")

Paramètres trouvés :
- Embed: 256
- Hidden: 512
- Vocab: 16434
Modèle prêt : vocab=16434, hidden=512, Unidirectionnel

 Modèle chargé et prêt !
Test génération : je suis désolé de vous avoir entraîné là-dedans


In [201]:
import time

def chat_with_model_secure(model):
    print("\n" + "="*60)
    print(" BOT LSTM AMÉLIORÉ (Tape 'quit' pour sortir)")
    print("   Note: Je complète tes phrases. Essaie de commencer une histoire.")
    print("="*60)
    
    model.eval()
    
    while True:
        try:
            user_input = input("\nToi (début de phrase) : ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("Bot: À la prochaine !")
                break
                
            if not user_input:
                continue

            # --- VÉRIFICATION 1 : Mots inconnus ---
            # On vérifie si les mots de l'utilisateur existent dans le vocabulaire
            words = user_input.lower().split()
            unknown_words = []
            for w in words:
                # On nettoie la ponctuation basique pour vérifier
                clean_w = w.replace("'", "' ") 
                if clean_w not in model.word_to_idx and w not in model.word_to_idx:
                    # Vérif un peu lâche pour les apostrophes
                    if clean_w.split()[0] not in model.word_to_idx:
                        unknown_words.append(w)
            
            if unknown_words:
                print(f"Bot: Désolé, je n'ai jamais appris ces mots : {unknown_words}")
                print("     (Mon vocabulaire est limité à ~2000 mots courants)")
                continue

            # --- VÉRIFICATION 2 : Longueur ---
            if len(words) < 2 and user_input.lower() not in ["je", "tu", "il", "elle", "nous", "vous", "ils", "elles", "c'est"]:
                print("Bot: C'est un peu court... Donne-moi au moins un sujet et un verbe.")
                continue

            print("Bot (écrit...)", end="\r")
            
            # On utilise generate_best pour éviter le charabia
            response = model.generate_best(
                user_input, 
                n_candidates=10, 
                max_length=15, 
                temperature=0.6, # Température moyenne pour éviter le délire total
                top_k=20
            )
            
            # Affichage propre : on montre la phrase complète
            print(f"Bot complète : \"{response}\"")
                
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"Erreur : {e}")

# Lancer la version sécurisée
chat_with_model_secure(loaded_model)


 BOT LSTM AMÉLIORÉ (Tape 'quit' pour sortir)
   Note: Je complète tes phrases. Essaie de commencer une histoire.
Bot: À la prochaine !
